In [1]:

import csv
import os
import sys
from docplex.mp.progress import *
from docplex.mp.progress import SolutionRecorder
import docplex.mp.model as cpx
import networkx as nx
import time
import random
from config import get_config
from network import *
from work_load import *

In [10]:
class Solver:
    def __init__(self):
        pass
    def request_service_delay_minimization(self,network,work_load,life_time,iteration,cyclic_workload,storage_capacity,delat_value):        

        import docplex.mp.model as cpx
        opt_model = cpx.Model(name="Storage problem model"+str(iteration))
        w_vars = {}
        u_vars = {}

        w_vars  = {(t,k,p): opt_model.continuous_var(lb=0, ub= network.max_edge_capacity,
                                  name="w_{0}_{1}_{2}".format(t,k,p))  for t in work_load.T 
                   for k in work_load.each_t_requests[t] 
                   for p in network.each_request_real_paths[k]+network.each_request_virtual_paths[k]}

        u_vars  = {(t,j,b): opt_model.continuous_var(lb=0, ub= network.max_edge_capacity,
                                      name="u_{0}_{1}_{2}".format(t,j,b))  for t in work_load.T 
                       for j in network.storage_pairs for b in network.each_storage_blocks[j] 
                   }   

        if life_time ==1000:
            #inventory evolution constraint
            for t in work_load.T[1:]:
                for j in network.storage_pairs:
                    for b in network.each_storage_blocks[j]:
#                         for p_s in network.each_request_real_paths[j,b]:
                        if cyclic_workload:
                            print("testing",network.each_storage_block_freshness(j,b,delat_value))
                            opt_model.add_constraint(u_vars[t,j,b] == u_vars[(t-1)%len(work_load.T),j,b]/network.each_storage_block_freshness(j,b,delat_value)-
                            opt_model.sum(w_vars[(t-1)%len(work_load.T),k,p] *
                            network.get_required_purification_EPR_pairs(p,work_load.get_each_request_threshold(k,t))
                            for k in work_load.each_t_requests[t] if k!=j 
                            for p in network.each_request_virtual_paths_using_block[k,j,b])*delat_value
                            +opt_model.sum(w_vars[(t-1)%len(work_load.T),j,p2] for p2 in network.each_storage_block_paths[j][b])*delat_value
                                                 ,ctname="inventory_evolution_{0}_{1}_{2}".format(t,j,b))
#                         else:
#                             opt_model.add_constraint(u_vars[t,j,b,p_s] == u_vars[t-1,j,b,p_s]/network.each_storage_block_freshness(j,b,delat_value)-
#                             opt_model.sum(w_vars[t-1,k,p] *
#                             network.get_required_purification_EPR_pairs(p,work_load.get_each_request_threshold(k,t))
#                             for k in work_load.each_t_requests[t] if k!=j 
#                             for p in network.each_request_virtual_paths_include_subpath[k][p_s])*delat_value
#                             +opt_model.sum(w_vars[t-1,j,p_s])*delat_value
#                                                  , ctname="inventory_evolution_{0}_{1}".format(t,j,p_s))
        else:
            #inventory evolution constraint
            for t in work_load.T[1:]:
                for j in network.storage_pairs:
                    for b in network.each_storage_blocks[j]:
                        for p_s in network.each_request_real_paths[j,b]:

                            if cyclic_workload:
                                opt_model.add_constraint(u_vars[t,j,b,p_s] == -
                                opt_model.sum(w_vars[(t-1)%len(work_load.T),k,p] *
                                network.get_required_purification_EPR_pairs(p,work_load.get_each_request_threshold(k,t))
                                for k in work_load.each_t_requests[t] if k!=j 
                                for p in network.each_request_virtual_paths_include_subpath[k][p_s] 
                                )*delat_value
                                + opt_model.sum(w_vars[(t-1)%len(work_load.T),j,p_s])*delat_value
                                                     , ctname="inventory_evolution_{0}_{1}".format(t,j,p_s))
                            else:
                                opt_model.add_constraint(u_vars[t,j,b,p_s] == -
                                opt_model.sum(w_vars[t-1,k,p] *
                                network.get_required_purification_EPR_pairs(p,work_load.get_each_request_threshold(k,t))
                                for k in work_load.each_t_requests[t] if k!=j 
                                for p in network.each_request_virtual_paths_include_subpath[k][p_s] 
                                )*delat_value
                                + opt_model.sum(w_vars[t-1,j,p_s])*delat_value
                                                     , ctname="inventory_evolution_{0}_{1}".format(t,j,p_s))

        # serving from inventory constraint
        for t in work_load.T[1:]:
            for j in network.storage_pairs:
                for b in network.each_storage_blocks[j]:
                    opt_model.add_constraint(opt_model.sum(w_vars[t,k,p]*
                    network.get_required_purification_EPR_pairs(p,work_load.get_each_request_threshold(k,t))
                    for k in work_load.each_t_user_pairs[t] if k!=j 
                    for p in network.each_request_virtual_paths_using_block[k,j,b]
                    )*delat_value<=u_vars[t,j,b]
                                         , ctname="inventory_serving_{0}_{1}_{2}".format(t,j,b))  


        # Demand constriant
        for t in work_load.T[1:]:
            for k in  work_load.each_t_requests[t]:
                opt_model.add_constraint(opt_model.sum(w_vars[t,k,p]
                for p in network.each_request_real_paths[k]+network.each_request_virtual_paths[k]) >= 
                        work_load.each_t_each_request_demand[t][k], ctname="constraint_{0}_{1}".format(t,k))

        #Edge constraint
        for t in work_load.T:
            for edge in network.set_E:
                opt_model.add_constraint(
                    opt_model.sum(w_vars[t,k,p]*network.get_required_purification_EPR_pairs(p,work_load.get_each_request_threshold(k,t)) for k in work_load.each_t_requests[t]
                    for p in network.each_request_real_paths[k]+network.each_request_virtual_paths[k] if network.check_path_include_edge(edge,p))

                     <= network.each_edge_capacity[edge], ctname="edge_capacity_{0}_{1}".format(t,edge))

        # storage servers capacity constraint
    #     storage_capacity = storage_capacity/delat_value
        for t in work_load.T:
            #for s1 in network.storage_nodes:
            for j in network.storage_pairs:
        
                opt_model.add_constraint(opt_model.sum(u_vars[t,j,b]
                    for b in network.each_storage_blocks[j]) <= storage_capacity 
            , ctname="storage_capacity_constraint_{0}_{1}".format(t,j))

        # constraints for serving from storage at time zero and 1 should be zero
#         if not cyclic_workload:
#             for t in [0,1]:
#                 opt_model.add_constraint(opt_model.sum(w_vars[t,k,p]
#                         for k in work_load.each_t_requests[t] for p in network.each_request_virtual_paths[k] 
#                         )<=0, ctname="serving_from_inventory_{0}".format(t))

#         # constraints for putting in storage at time zero  should be zero
#         """this is becasue we start the formulation from 1 and not from zero and we have t-1 in our formulation"""
#         for t in [0]:
#             opt_model.add_constraint(opt_model.sum(w_vars[t,k,p]
#                     for k in work_load.each_t_requests[t] for p in network.each_request_real_paths[k] 
#                     )<=0, ctname="storing_in_inventory_{0}".format(t))   


#         # constraint for inventory is zero at time zero 
#         if not cyclic_workload:
#             for t in [0]:
#                 for j in network.storage_pairs:
#                      for p_s in network.each_request_real_paths[j]:
#                             opt_model.add_constraint(u_vars[t,j,p_s] <=0, ctname="storage_capacity_constraint_{0}_{1}_{2}".format(t,j,p_s))

        """defining an objective, which is a linear expression"""

        objective = opt_model.sum(1/len(work_load.T[1:])*1/len(work_load.each_t_real_requests[t])*1/work_load.each_t_each_request_demand[t][k]
                                  *(w_vars[t,k,p] * network.get_path_length(p)) for t in work_load.T[1:]
                                  for k in work_load.each_t_user_pairs[t] 
                                  for p in network.each_request_real_paths[k]+network.each_request_virtual_paths[k]
                                  )



        opt_model.minimize(objective)

        opt_model.print_information()

        opt_model.solve()


        print('docplex.mp.solution',opt_model.solution)
        objective_value = -1
        try:
            if opt_model.solution:
                objective_value =opt_model.solution.get_objective_value()
        except ValueError:
            print(ValueError)
        each_inventory_per_time_usage = {}
        each_time_each_path_delivered_EPRs = {}
        each_time_each_path_purification_EPRs = {}
        each_time_used_from_storage_EPR_pairs_for_purification = []
        each_time_delivered_from_storage_EPR_pairs = []
        all_times_all_storages_stored_EPR_pairs = []
        if objective_value >0:
            if opt_model.solution:
                print("this has been solved! iteration",iteration)
            print('docplex.mp.solution',opt_model.solution)
            #print("******************************** start *************************************")
            for t in work_load.T[1:]:
                for j in network.storage_pairs:

                    for p in network.each_request_real_paths[j]:
                        usage_of_storage_pair = 0
                        usage_of_storage_pair+=u_vars[t,j,p].solution_value
                        #if u_vars[t,j,p].solution_value>0:
                            #print("u_%s_%s_%s=%s"%(t,j,p,u_vars[t,j,p].solution_value))
                        try:
                            each_inventory_per_time_usage[j][t]=u_vars[t,j,p].solution_value
                        except:
                            try:
                                each_inventory_per_time_usage[j][t]=u_vars[t,j,p].solution_value
                            except:
                                each_inventory_per_time_usage[j] = {}
                                each_inventory_per_time_usage[j][t]=u_vars[t,j,p].solution_value        
                        all_times_all_storages_stored_EPR_pairs.append(usage_of_storage_pair)

            #print("******************************** end *************************************")   

            for t in work_load.T:
                u_value = 0
                for j in network.storage_pairs:
                    for p in network.each_request_real_paths[j]:
                        if storage_capacity< int(u_vars[t,j,p].solution_value)-1:
                            print("THIS IS a BIG ERROR!!!",u_vars[t,j,p].solution_value , storage_capacity)
                            import pdb
                            pdb.set_trace()


                    #if u_value > storage_capacity:
                        #print("Error!!!!!")
                        #print("u value for storage %s is %s  and we have capacity as %s"%(s1,u_value,storage_capacity))
                        #import pdb
                        #pdb.set_trace()
                    #else:
                        #print("OK!")


        if objective_value>0:
            for t in work_load.T[1:]:
                for k in work_load.each_t_real_requests[t]: 
                    for p in network.each_request_virtual_paths[k]:
                        try:
                            each_time_delivered_from_storage_EPR_pairs.append(w_vars[t,k,p].solution_value)
                        except:
                            each_time_delivered_from_storage_EPR_pairs=[w_vars[t,k,p].solution_value]
                        purification_EPR_pairs = network.get_required_purification_EPR_pairs(p,work_load.get_each_request_threshold(k,t))
                        try:
                            each_time_used_from_storage_EPR_pairs_for_purification.append(purification_EPR_pairs)
                        except:
                            each_time_used_from_storage_EPR_pairs_for_purification=[purification_EPR_pairs]
            for t in work_load.T[1:]:
                for k in work_load.each_t_real_requests[t]: 
                    for p in network.each_request_real_paths[k]+network.each_request_virtual_paths[k]:
                        try:
                            each_time_each_path_delivered_EPRs[t][k]+=w_vars[t,k,p].solution_value
                        except:
                            try:
                                each_time_each_path_delivered_EPRs[t][k]= w_vars[t,k,p].solution_value
                            except:
                                each_time_each_path_delivered_EPRs[t]={}
                                each_time_each_path_delivered_EPRs[t][k]= w_vars[t,k,p].solution_value

                        try:
                            each_time_each_path_purification_EPRs[t][k]+=network.get_required_purification_EPR_pairs(p,work_load.get_each_request_threshold(k,t))
                        except:
                            try:
                                each_time_each_path_purification_EPRs[t][k]= network.get_required_purification_EPR_pairs(p,work_load.get_each_request_threshold(k,t))
                            except:
                                each_time_each_path_purification_EPRs[t]={}
                                each_time_each_path_purification_EPRs[t][k]= network.get_required_purification_EPR_pairs(p,work_load.get_each_request_threshold(k,t))        

        opt_model.clear()
        return objective_value,all_times_all_storages_stored_EPR_pairs,each_time_delivered_from_storage_EPR_pairs,each_time_used_from_storage_EPR_pairs_for_purification



    
    
    

In [11]:
#paths: 0:(0,1),(1,2),(2,3),(3,4),(4,5),
        #1:(1,2),(2,3),(3,4),
        #2:(0,1),(1,4),(4,5)
        

network = Network()
network.set_of_paths = {0:[0,1,2,3,4],1:[1,2,3],2:[0,5]}


network.max_edge_capacity =100
network.fidelity_threshold_values = [0.9,0.94,0.95,0.98]

network.each_path_basic_fidelity  ={0:0.8,1:0.9,2:0.8}
network.oracle_for_target_fidelity = {}
network.set_required_EPR_pairs_for_each_path_each_fidelity_threshold()


network.each_request_real_paths = {0:[0]}
network.each_request_virtual_paths_using_block = {0:{0:{0:1}}}


network.each_request_virtual_paths = {0:[1]}
network.storage_pairs = [0]
network.each_storage_blocks ={0:[0]} 
network.each_request_real_paths = {0:[1]}
network.each_storage_block_paths = {0:{0:[1]}}

#Edge constraint
network.set_E = [0,1,2,3,4,5]
network.each_edge_capacity = {0:100,1:100,2:100,3:100,4:100,5:100}

storage_capacity  = 500
          
    
    
work_load = Work_load()
work_load.each_t_user_pairs = {0:[0],1:[0],2:[0],3:[0]}
work_load.T  = [0,1,2,3]
work_load.each_t_requests = {0:[0],1:[0],2:[0],3:[0],4:[0]} 
# Demand constriant
work_load.each_t_each_request_demand = {0:{0:5},1:{0:20},
                                        2:{0:5},3:{0:10}}    
work_load.each_request_threshold = {0:{0:0.9,1:0.9,2:0.9,3:0.9}}
work_load.each_t_real_requests = {0:[0],1:[0],2:[0],3:[0],4:[0]}

[1]


In [12]:
solver =Solver()
solver.request_service_delay_minimization(network,work_load,
                                          1000,0,True,100,10)



testing 1
testing 1
testing 1
Model: Storage problem model0
 - number of variables: 12
   - binary=0, integer=0, continuous=12
 - number of constraints: 37
   - linear=37
 - parameters: defaults
 - objective: minimize


DOcplexException: Cannot solve model: no CPLEX runtime found.